# Demonstrating the reusability of notebooks.

Since notebooks are json they may be considered to be data.  This project loads in and explores notebooks as dataframes.

* `importnb` imports notebooks as modules
* `importnb` allows notebooks to be discovered as tests.

    >>> with Notebook(): import notebook_dir as nb
    >>> assert 'test_function' in dir(nb)
    >>> assert nb.notebook_directory_to_df()

# This notebooks loads in a directory of notebooks as dataframe.

In [1]:
    from toolz.curried import *
    from toolz.curried.operator import *
    from pathlib import Path
    from pandas import Series, DataFrame, concat as Concat

    ignores = 'checkpoint',

    from nbformat.v4 import reads

    def notebook_directory_to_df(location=''):
        """Convert a directory of notebooks in dataframes.

        returns a dataframe for the cells and for the metadata.
        """
        cells = metadata = None

        suffix = '.ipynb'
        recursive = False
        for path in getattr(Path(location), 'glob')(f"*{recursive and 'r' or ''}{suffix}"):
            for ignore in ignores:
                if ignore in str(path): continue

            try:
                nb = pipe(path.read_text(), reads)
            except: continue

            s = DataFrame({k: v for k, v in nb.items() if k !='cells'})
            s = s.metadata.apply(Series).stack().append(Concat({'nbformat': s.iloc[0][['nbformat', 'nbformat_minor']]}))

            if cells is None:
                cells = Concat({path: DataFrame(nb['cells'])})
                metadata = Concat({path: s})
            else:
                cells = cells.append(Concat({path: DataFrame(nb['cells'])}))
                metadata = metadata.append(Concat({path: s}))
        return cells, metadata.unstack([-2, -1])

In [2]:
    __all__ = 'test_function',

# ipywidgets

In [3]:
from traitlets import dlink
from ipywidgets import Text, HTML, VBox
from functools import partial

df = None
def finder(query, dir=''):
    global df
    if df is None:
        df, metadata = notebook_directory_to_df(dir)
        globals()['df'] = df
    return df[df.source.apply(''.join).apply(lambda x: query in x)].to_html()
    
;

def create_query_widget(location=''):
    """Create a widget to query a dataframe of notebooks.
    
        create_query_widget('')
        
    """
    query = Text(description='Find cells containing a string.')
    table = HTML()
    dlink((query, 'value'), (table, 'value'), partial(finder, dir=location))
    return VBox(children=[query, table])

> ### [`interact`](http://ipywidgets.readthedocs.io/en/latest/examples/Using%20Interact.html) is the simplest use of widgets.

## pytest will discover `test_function` because it starts with `test_*`

In [4]:
    def test_function():
        assert True

## doctest can test the current notebook state.

In [5]:
    if __name__ == '__main__':
        import doctest
        doctest.testmod()

## doctest can test the future state of the module.

In [6]:
    if __name__ == '__main__':
        with Notebook():
            import test_notebook_dir as nb
            doctest.testmod(nb)

NameError: name 'Notebook' is not defined